爬蟲思考順序  
S1: 找到我們要的資訊 (圖or文字資訊等等...) 在哪裡  
&emsp;1.1 右鍵 -> 檢視原始碼 -> 有，網址列url -> 待解析的資訊  
&emsp;1.2 沒有 -> 開發人員工具 -> Network 標頭，url -> 待解析的資訊  

In [1]:
from urllib.request import urlopen   # 從網址列問問題的模組

url= "https://www.google.com/doodles/json/2022/12?hl=zh-TW"   # 網址
response= urlopen(url)   # 對方伺服器回應的答案

response.read()   # 讀取該答案 - 但此回應人類看不懂

b'[{"name": "new-years-eve-2022", "title": "2022 \\u5e74\\u8de8\\u5e74\\u591c", "url": "//www.google.com/logos/doodles/2022/new-years-eve-2022-6753651837109565-law.gif", "alternate_url": "https://lh3.googleusercontent.com/8moaWCQ_NZMv7yDFP_Nr8rBjtUhROn9nQrUiuDr_EXwGPRno09xlQZ7k6mM2JJy2ACB4tdWZl6eJi19YhMBECNeucNBwbXeOG3an94xuBm2Hm9mF5eIsdjg", "high_res_url": "//www.google.com/logos/doodles/2022/new-years-eve-2022-6753651837109565-2xa.gif", "high_res_width": 1000, "high_res_height": 400, "run_date_array": [2022, 12, 31], "translations": {"en": {"query": "New Year\'s Eve 2022", "hover_text": "New Year\'s Eve 2022", "share_text": "New Year\'s Eve 2022 #GoogleDoodle"}, "af": {"query": "Oujaarsaand 2022", "hover_text": "Oujaarsaand 2022", "share_text": "Oujaarsaand 2022 #GoogleDoodle"}, "sq": {"query": "Pragu i Vitit t\\u00eb Ri 2022", "hover_text": "Pragu i Vitit t\\u00eb Ri 2022", "share_text": "Pragu i Vitit t\\u00eb Ri 2022 #GoogleDoodle"}, "am": {"query": "\\u12e8\\u12a0\\u12f2\\u1235 \

伺服器的回應類型  
S2: 不同的回應類型，我們需要用不同的解析手段  
&emsp;1.1 回應1: 網頁  
&emsp;1.2 回應2: 多媒體&emsp;<-&emsp;read  
&emsp;1.3 回應3: json   ( [] , {} )&emsp;<-&emsp;load

In [ ]:
from urllib.request import urlopen

url= "https://www.google.com/doodles/json/2022/12?hl=zh-TW"
response= urlopen(url)

import json
# json有兩種解析模式 json.load(原始檔案) / json.loads(限字串)
doodles= json.load(response)   # list

for d in doodles:   # 取出list中，每個字典中，title的值 / url的值
    url= "https:" + d["url"]
    print(d["title"])
    print(url)

    # 拿到圖片的網址了，開始處理圖片
    response= urlopen(url)   # 此回應是多媒體
    img= response.read()
    fname= "D:/Coding_Program/Tibame_Python/doodles/" + url.split("/")[-1]   # 檔名，直接用網址的最後一串字元，split -> list -> 取[-1]
    
    # 存檔三部曲 open(圖檔不用encoding) -> write -> close
    f= open(fname, "wb")
    f.write(img)
    f.close()

模組 urlretrive


In [ ]:
from urllib.request import urlopen, urlretrieve
import json

url= "https://www.google.com/doodles/json/2023/1?hl=zh-TW"
response= urlopen(url)

doodles= json.load(response)

for d in doodles:
    url= "https:" + d["url"]
    print(d["title"])
    print(url)

    fname= "D:/Coding_Program/Tibame_Python/doodles/" + url.split("/")[-1]
    urlretrieve(url, fname)   # 網址, 檔名 會自動:開啟，寫入，關閉 圖檔

迴圈，os -> mkdir

In [ ]:
from urllib.request import urlopen, urlretrieve
import json
import os

for page in range(1,13):
    url= f"https://www.google.com/doodles/json/2022/{page}?hl=zh-TW"
    response= urlopen(url)

    page= json.load(response)
    for p in page:
        url= "https:" + p["url"]
        print(p["title"])
        print(url)

        dirname = "D:/Coding_Program/Tibame_Python/doodles/"+ str(page) +"/"

        if not os.path.exists(dirname):
            os.mkdir(dirname)
        
        fname= dirname + url.split("/")[-1]
        urlretrieve(url,fname)

伺服器的回應類型-2  
S2: 不同的回應類型，我們需要用不同的解析手段  
&emsp;1.1 回應1: 網頁&emsp;<-BeautifulSoup&emsp;<-  find, find_all  
&emsp;1.2 回應2: 多媒體&emsp;<-&emsp;read  
&emsp;1.3 回應3: json   ( [] , {} )&emsp;<-&emsp;load

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError

page= 55     # 最後增加迴圈時，發現超過頁數，會return HTTPError: HTTP Error
while True:
    url= f"https://tabelog.com/tw/tokyo/rstLst/{page}/?SrtT=rt"
    try:
        response= urlopen(url)
    except HTTPError:
        print("這裡是最後一頁")
        break

    html= BeautifulSoup(response,"html.parser")

    for r in html.find_all("li", class_="list-rst"):   # li 是大盒子，可以在裡面找其他所需元素
        jp= r.find("small", class_="list-rst__name-ja")
        en= r.find("a", class_="list-rst__name-main")
        link= en["href"]
        rate= r.find("b", class_= "c-rating__val")
        n_price= r.find_all("span", class_="c-rating__val")[0]
        d_price= r.find_all("span", class_="c-rating__val")[1]
        print(f"日:{jp.text},英:{en.text},評分:{rate.text}")
        print(f"日間價位:{d_price.text}, 夜間價位:{n_price.text}")
        print(f"Link:{link}")
    
    page= page+ 1

爬蟲實作1:PTT  
(requests, extract, jieba)

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

url= "https://www.ptt.cc/bbs/movie/M.1677872664.A.259.html"
rep= Request(url,headers={"User-Agent":"Mozilla"})

response= urlopen(rep)
html= BeautifulSoup(response,"html.parser")   # error 403，需要模擬正常使用者請求

html   # 用下方 requests 繼續練習

In [ ]:
import requests
from bs4 import BeautifulSoup
from jieba.analyse import extract_tags

url= "https://www.ptt.cc/bbs/movie/M.1677872664.A.259.html"
headers= {"User-Agent":"Mozilla"}   # 有時requests的headers要自己調整

response= requests.get(url,headers= headers)   # 調整headers
html= BeautifulSoup(response.text,"html.parser")   # 記得response.text

# PTT 文字直接放在maintain下方，並不屬於任何盒子，所以先取全部，再一一刪除

content= html.find("div", id= "main-content")
# 移除作者
deleted_content= content.find_all("div", class_= "article-metaline")
for d in deleted_content:
    d.extract()
# 移除看板
deleted_content= content.find_all("div", class_= "article-metaline-right")
for d in deleted_content:
    d.extract()
# 移除文章網址
deleted_content= content.find_all("span", class_= "f2")
for d in deleted_content:
    d.extract()
# 移除推噓文   (如果推噓文有多個class_，可以選擇較有意義的class_使用)
deleted_content= content.find_all("div", class_= "push")
for d in deleted_content:
    d.extract()
# 移除文章中第二個超連結
deleted_content= content.find_all("a")[1]
for d in deleted_content:
    d.extract()

key_words= extract_tags(content.text,3)

print(content.text)
print(f"關鍵字: {key_words}")

爬蟲實作2:PTT八卦版  
設置cookie

In [ ]:
import requests
from bs4 import BeautifulSoup

url= "https://www.ptt.cc/bbs/Gossiping/M.1677120107.A.596.html"
response= requests.get(url,cookies={"over18":"1"})

html= BeautifulSoup(response.text, "html.parser")

content= html.find("div", id="main-content")

deleted_content= content.find_all("div", class_= "article-metaline")
for d in deleted_content:
    d.extract()

deleted_content= content.find_all("div", class_= "article-metaline-right")
for d in deleted_content:
    d.extract()

deleted_content= content.find_all("div", class_= "push")
for d in deleted_content:
    d.extract()

deleted_content= content.find_all("span", class_= "f2")
for d in deleted_content:
    d.extract()

print(content.text)

爬蟲的資料儲存  
pandas

In [3]:
import pandas as pd
df= pd.read_csv("titantic.csv", encoding= "UTF-8")
#df["comments"]   # 單行操作
#df[ ["comments", "name"] ]   # 多行操作 (外[]->df / 內[]->list)
#df.T   # 翻轉檔案

In [ ]:
# 列操作
df.head(5)
df.tail(5)
df.iloc[1000:1005][ ["comments", "name"] ]

In [ ]:
# 更新df : 轉換流程 -> 傳送回df -> 更新原本df
from datetime import datetime

# 轉換流程
def timetrans(s):
    return str(datetime.utcfromtimestamp(s))

# apply 會將我方定義的流程，自動帶入 df 每列資料
df["film_date"].apply(timetrans)

# 傳送回df
df["new_film_date"]= df["film_date"].apply(timetrans)   # 新增成功
df

In [ ]:
# 篩選操作   (輸入跟列一樣多的布林，True會被留下)
# 思考順序 -> 需要布林列表 -> 使用apply去判斷True/False -> 對apply出的新欄位篩選

df.head(4)[ [True, False, True, False] ]   # 0 2 列

# 篩選 comments > 3000 的列資料
def greater3000(s):
    if s > 3000:
        return True
    else:
        return False

df["comments"].apply(greater3000)
df["c_judge"]= df["comments"].apply(greater3000)
df[ list(df["c_judge"]) ]

# 也可以不用新增欄位
t= df["comments"].apply(greater3000)
df[t]

In [44]:
# tags 中，含有特定資料的列
def has(s):
    if "children" in s and "culture" in s:   # 多條件篩選
        return True
    else:
        return False

test= df["tags"].apply(has)
df[ ["comments", "tags"] ][test]

# 小提醒:tags 看似list，但其實屬性為有 [] 的字串
# 新function eval()
# 把傳入的字串，當成一個python 程式運行
def fflow(s):
    taglist= eval(s)   # 轉化成 List
    if "children"in taglist:
        return True
    else:
        return False

test1= df["tags"].apply(fflow)
df[ ["comments", "tags"] ][test1]

# 儲存為csv檔案   -> 儲存前，需取代原本df or 另外新增一個df
newdf= df[ ["comments", "tags"] ][test1]
newdf.to_csv("測試檔案.csv", index=False)

In [ ]:
# 將爬蟲資訊，放入pandas

# 思考順序
"""
    df= pd.DataFrame(columns= [])                    # 創造空的df 並自己命名欄位名稱
    s= pd.Series [(欄位), index= [我們命名的欄位]]    # 帶入我們爬出的資料
    df.append(s,ignore_index=True)                   # 忽略default index
    df.to_csv("", encoding, index)                   # 存檔
"""
# tabelog 練習
import requests
from bs4 import BeautifulSoup
from urllib.request import HTTPError
import pandas as pd

df= pd.DataFrame(columns=["JA", "EN", "Dprice", "Nprice", "url"])

page= 55
while page<58:
    url= f"https://tabelog.com/tw/tokyo/rstLst/{page}/?SrtT=rt"
    try:
        response= requests.get(url)
    except HTTPError:
        print("last page")

    html= BeautifulSoup(response.text, "html.parser")

    for t in html.find_all("li", class_= "list-rst"):
        ja= t.find("small", class_= "list-rst__name-ja")
        en= t.find("a", class_="list-rst__name-main")
        url= en["href"]
        n_price= t.find_all("span", class_="c-rating__val")[0]
        d_price= t.find_all("span", class_="c-rating__val")[1]

        s= pd.Series([ja.text, en.text, d_price.text, n_price.text, url],
                        index= ["JA", "EN", "Dprice", "Nprice", "url"])
        df= df.append(s,ignore_index=True)   # 是否要忽略每個Series自帶的index
    page= page+ 1

df.to_csv("3_pages.csv", encoding= "UTF-8", index= False)   # to_csv，增加一欄index?

Matplotlib

In [ ]:
import matplotlib.pyplot as plt

df= pd.read_csv("titantic.csv", encoding= "UTF-8")

plt.hist(df["languages"], range= (0,70), bins= 25)   # X軸，min-max / 資料分成 bins 份
plt.show()